In [46]:
# Import modules
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps
import pyproj

In [47]:
# Load Sampled Data
df=pd.read_csv("../data/Simplified Citation After 2015 July to 2019.csv",low_memory=False)
df.head(5)

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.419487e+06,1.818861e+06
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.473690e+06,1.808839e+06
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.475329e+06,1.834194e+06
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.417752e+06,1.820642e+06
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.459495e+06,1.857205e+06


In [48]:
df.columns

Index(['Unnamed: 0', 'Ticket number', 'Issue Date', 'Issue time', 'Meter Id',
       'Marked Time', 'RP State Plate', 'VIN', 'Make', 'Body Style', 'Color',
       'Location', 'Route', 'Agency', 'Violation code',
       'Violation Description', 'Fine amount', 'Latitude', 'Longitude'],
      dtype='object')

In [49]:
# Identify bad Longitude/Latitude values
df['Longitude'].value_counts()

9.999900e+04    11416
1.819688e+06       98
1.859071e+06       87
1.882602e+06       64
1.849114e+06       49
1.849114e+06       46
1.859071e+06       44
1.836817e+06       44
1.849114e+06       42
1.876409e+06       42
1.819197e+06       41
1.837270e+06       41
1.864752e+06       41
1.839201e+06       40
1.883364e+06       40
1.845451e+06       38
1.852081e+06       38
1.849337e+06       36
1.857542e+06       35
1.848423e+06       35
1.848423e+06       35
1.840434e+06       35
1.857542e+06       34
1.845113e+06       34
1.845113e+06       34
1.889882e+06       34
1.845233e+06       33
1.858834e+06       33
1.882604e+06       33
1.858229e+06       32
                ...  
1.842851e+06        1
1.807688e+06        1
1.844638e+06        1
1.903358e+06        1
1.844035e+06        1
1.879962e+06        1
1.823354e+06        1
1.803389e+06        1
1.840971e+06        1
1.851902e+06        1
1.847878e+06        1
1.841124e+06        1
1.841152e+06        1
1.842144e+06        1
1.850810e+

In [50]:
# Fraction of records with bad Lat/Lon values
len(df[(df['Latitude'] == 99999) | (df['Longitude'] == 99999)])/len(df)

0.13038810334193754

In [51]:
# Filter and create cleaned dataset
df = df[(df['Latitude'] != 99999) | (df['Longitude'] != 99999)]

In [52]:
# New dataset statistics
df[['Latitude', 'Longitude']].describe()

,Latitude,Longitude
count,7.613800e+04,7.613800e+04
mean,6.455758e+06,1.848533e+06
std,2.734972e+04,2.866901e+04
min,6.360390e+06,1.715644e+06
25%,6.431560e+06,1.837057e+06
50%,6.458808e+06,1.845113e+06
75%,6.478079e+06,1.859516e+06
max,6.513150e+06,1.941866e+06


In [53]:
# Conversion using pyproj module
inProj = pyproj.Proj('epsg:2229', preserve_units=True) # make sure unit preserve 
outProj = pyproj.Proj('epsg:4326')
y_1,x_1 = pyproj.transform(inProj,outProj,df["Latitude"].values,df["Longitude"].values)

In [54]:
# Add on the converted coordinates
df["Latitude"] = y_1
df["Longitude"] = x_1
df.head()

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472


In [55]:
# Check for null values
df.isnull().sum()

Unnamed: 0                   0
Ticket number                0
Issue Date                   0
Issue time                  11
Meter Id                 54920
Marked Time              73582
RP State Plate               1
VIN                      76138
Make                        31
Body Style                  40
Color                       12
Location                     0
Route                      331
Agency                       0
Violation code               0
Violation Description        2
Fine amount                 23
Latitude                     0
Longitude                    0
dtype: int64

In [56]:
df['Violation code'][df['Violation Description'] == 'NO PARK/STREET CLEAN'].value_counts()

80.69BS    21208
8069BS       491
Name: Violation code, dtype: int64

In [57]:
df['Violation Description'].unique()

array(['NO PARK/STREET CLEAN', 'EXCEED 72HRS-ST', 'RED ZONE',
       'NO STOP/STAND', 'STANDNG IN ALLEY', 'DISPLAY OF PLATES',
       'METER EXP.', 'DISPLAY OF TABS', 'PARKED ON SIDEWALK',
       'FIRE HYDRANT', 'NO PARKING', 'WHITE ZONE', 'RESTRICTED TAXI ZONE',
       'DISABLED PARKING/NO DP ID', 'NO STOPPING/ANTI-GRIDLOCK ZONE',
       'PARKED OVER TIME LIMIT', 'NO STOP/STANDING',
       'BLK BIKE PATH OR LANE', 'OUTSIDE LINES/METER',
       'OVNIGHT PRK W/OUT PE', 'PREFERENTIAL PARKING', 'YELLOW ZONE',
       'DOUBLE PARKING', 'PARKED IN BUS ZONE', 'HANDICAP/NO DP ID',
       'STOP/STAND PROHIBIT', 'PARKED IN PARKWAY', '22514',
       'BLOCKING DRIVEWAY', '18 IN. CURB/2 WAY',
       'NO PARKING BETWEEN POSTED HOURS', 'PRIVATE PROPERTY',
       'OFF STR MTR/OUT LINE', 'PARKED IN FIRE LANE',
       'COMM TRAILER/22 FT.', 'DISABLED PARKING/BOUNDARIES',
       'OFF STR/OVERTIME/MTR', 'DP-BLKNG ACCESS RAMP', 'PK OVERSIZ',
       'CARSHARE PARKING', 'SAFETY ZONE/CURB', 'PARKING/FRONT YAR

In [58]:
df[df['Violation Description'] == 'CLEANING VEH/STREET']

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
17695,1753653,4338918846,2018/09/05 12:00:00 AM,1221.0,NaN,NaN,CA,NaN,OTHR,PA,RD,980 MANZANITA ST,00446,54.0,80.74,CLEANING VEH/STREET,53.0,34.090388,-118.282836
53686,5359303,4291389071,2017/01/13 12:00:00 AM,853.0,NaN,NaN,CA,NaN,HOND,PA,SL,14565 LANARK ST,00300,53.0,80.74,CLEANING VEH/STREET,53.0,34.217941,-118.450313
55191,5509385,4298738312,2016/12/19 12:00:00 AM,1103.0,NaN,NaN,CA,NaN,FORD,CM,BK,2626 ALSACE AVE,00531,55.0,80.74,CLEANING VEH/STREET,53.0,34.032247,-118.355517


In [59]:
df[df['Violation code'] == '80.69BS']['Violation Description']

0        NO PARK/STREET CLEAN
1        NO PARK/STREET CLEAN
2        NO PARK/STREET CLEAN
3        NO PARK/STREET CLEAN
4        NO PARK/STREET CLEAN
5        NO PARK/STREET CLEAN
6        NO PARK/STREET CLEAN
8        NO PARK/STREET CLEAN
10       NO PARK/STREET CLEAN
11       NO PARK/STREET CLEAN
18       NO PARK/STREET CLEAN
20       NO PARK/STREET CLEAN
22       NO PARK/STREET CLEAN
24       NO PARK/STREET CLEAN
25       NO PARK/STREET CLEAN
30       NO PARK/STREET CLEAN
38       NO PARK/STREET CLEAN
41       NO PARK/STREET CLEAN
44       NO PARK/STREET CLEAN
47       NO PARK/STREET CLEAN
48       NO PARK/STREET CLEAN
49       NO PARK/STREET CLEAN
51       NO PARK/STREET CLEAN
57       NO PARK/STREET CLEAN
58       NO PARK/STREET CLEAN
60       NO PARK/STREET CLEAN
64       NO PARK/STREET CLEAN
66       NO PARK/STREET CLEAN
67       NO PARK/STREET CLEAN
68       NO PARK/STREET CLEAN
                 ...         
87391    NO PARK/STREET CLEAN
87393    NO PARK/STREET CLEAN
87397    N

In [60]:
df[df['Violation Description'] == 'NO PARK/STREET CLEAN']

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472
5,618,4360059804,2019/09/17 12:00:00 AM,1032.0,NaN,NaN,CA,NaN,CHEV,PA,BK,250 CORONADO ST S,00452,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.071393,-118.274392
6,679,4359986643,2019/09/17 12:00:00 AM,1054.0,NaN,NaN,CA,NaN,HOND,PA,WT,15211 KITTRIDGE ST,350A,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.190290,-118.464223
8,824,4359867912,2019/09/17 12:00:00 AM,1028.0,NaN,NaN,CA,NaN,MERC,PA,SL,1222 POINT VIEW ST,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.053774,-118.369244
10,1009,4359605530,2019/09/17 12:00:00 AM,1308.0,NaN,NaN,CA,NaN,KIA,PA,BK,2200 65TH PL W,00500,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.980008,-118.317712
11,1037,4359594411,2019/09/17 12:00:00 AM,1229.0,NaN,NaN,CA,NaN,SUBA,PA,GN,643 VISTA ST N,00475,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.082591,-118.352228


In [61]:
df[(df['Location'].str.contains('WINDWARD')) & (df['Violation Description'] == 'NO PARK/STREET CLEAN')]

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024
1370,139182,4358481536,2019/07/22 12:00:00 AM,1140.0,NaN,NaN,AL,NaN,CADI,PA,BK,77 WINDWARD AVE E,112R,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987515,-118.472533
2307,229728,4357259970,2019/07/01 12:00:00 AM,1110.0,NaN,NaN,NV,NaN,NISS,PA,RD,69 WINDWARD AVE,00136,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987456,-118.472630
6525,651005,4352300315,2019/04/09 12:00:00 AM,1200.0,NaN,NaN,CA,NaN,LEXS,PA,GY,66 WINDWARD AVE,00135,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987435,-118.472664
7280,726282,4351612661,2019/03/26 12:00:00 AM,1111.0,NaN,NaN,UT,NaN,HYUN,PA,BL,64 WINDWARD AVE,00132,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987420,-118.472689
8345,829154,4349764495,2019/03/05 12:00:00 AM,1109.0,NaN,NaN,CA,NaN,MAZD,PA,SL,56 WINDWARD AVE,00132,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987359,-118.472788
16726,1659384,4340345590,2018/09/25 12:00:00 AM,1138.0,NaN,NaN,CA,NaN,SUBA,PA,BK,56 WINDWARD AVE,161R,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987359,-118.472788
20207,2010426,4332648950,2018/07/17 12:00:00 AM,1231.0,NaN,NaN,CA,NaN,HOND,PA,WT,325 WINDWARD AVE,00134,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989715,-118.468564
22779,2270957,4333866143,2018/05/29 12:00:00 AM,1132.0,NaN,NaN,CA,NaN,DODG,PA,SL,36 WINDWARD AVE,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987207,-118.473037
35531,3536292,4318915086,2017/10/30 12:00:00 AM,1140.0,NaN,NaN,CA,NaN,TOYT,PA,WT,23 WINDWARD AVE,00135,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.987115,-118.473187


In [62]:
pd.Timestamp(df.loc[0, 'Issue Date'])

Timestamp('2019-09-17 00:00:00')

In [63]:
df.loc[:,'Issue Date'] = df['Issue Date'].apply(pd.Timestamp)

In [64]:
df

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019-09-17,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024
1,112,4361286823,2019-09-17,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126
2,224,4361159515,2019-09-17,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957
3,473,4361237226,2019-09-17,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775
4,592,4360129196,2019-09-17,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472
5,618,4360059804,2019-09-17,1032.0,NaN,NaN,CA,NaN,CHEV,PA,BK,250 CORONADO ST S,00452,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.071393,-118.274392
6,679,4359986643,2019-09-17,1054.0,NaN,NaN,CA,NaN,HOND,PA,WT,15211 KITTRIDGE ST,350A,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.190290,-118.464223
7,694,4359985136,2019-09-17,1057.0,NaN,NaN,CA,NaN,TOYT,PA,BL,2131 CANYON DR,00402,54.0,80.73.2,EXCEED 72HRS-ST,68.0,34.109339,-118.316927
8,824,4359867912,2019-09-17,1028.0,NaN,NaN,CA,NaN,MERC,PA,SL,1222 POINT VIEW ST,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.053774,-118.369244
9,897,4359725355,2019-09-17,138.0,NaN,NaN,CA,NaN,JEEP,PA,GY,1000 39TH ST W,00500,55.0,80.56E4+,RED ZONE,93.0,34.015481,-118.291537


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76138 entries, 0 to 87553
Data columns (total 19 columns):
Unnamed: 0               76138 non-null int64
Ticket number            76138 non-null int64
Issue Date               76138 non-null datetime64[ns]
Issue time               76127 non-null float64
Meter Id                 21218 non-null object
Marked Time              2556 non-null float64
RP State Plate           76137 non-null object
VIN                      0 non-null float64
Make                     76107 non-null object
Body Style               76098 non-null object
Color                    76126 non-null object
Location                 76138 non-null object
Route                    75807 non-null object
Agency                   76138 non-null float64
Violation code           76138 non-null object
Violation Description    76136 non-null object
Fine amount              76115 non-null float64
Latitude                 76138 non-null float64
Longitude                76138 non-nul

In [66]:
df['Issue Date'].apply(lambda x: x.dayofweek)

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       4
23       4
24       4
25       4
26       4
29       4
30       4
32       4
33       4
        ..
87523    3
87524    3
87525    3
87526    3
87527    3
87528    3
87529    3
87530    3
87531    3
87532    3
87533    3
87534    3
87535    3
87536    3
87537    3
87538    3
87540    3
87541    3
87542    3
87543    3
87544    3
87545    3
87546    3
87547    3
87548    3
87549    3
87550    3
87551    3
87552    3
87553    3
Name: Issue Date, Length: 76138, dtype: int64

In [67]:
df.loc[:,'Day of Week'] = df['Issue Date'].apply(lambda x: x.dayofweek)

In [68]:
df

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Day of Week
0,28,4361326712,2019-09-17,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024,1
1,112,4361286823,2019-09-17,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126,1
2,224,4361159515,2019-09-17,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957,1
3,473,4361237226,2019-09-17,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775,1
4,592,4360129196,2019-09-17,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472,1
5,618,4360059804,2019-09-17,1032.0,NaN,NaN,CA,NaN,CHEV,PA,BK,250 CORONADO ST S,00452,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.071393,-118.274392,1
6,679,4359986643,2019-09-17,1054.0,NaN,NaN,CA,NaN,HOND,PA,WT,15211 KITTRIDGE ST,350A,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.190290,-118.464223,1
7,694,4359985136,2019-09-17,1057.0,NaN,NaN,CA,NaN,TOYT,PA,BL,2131 CANYON DR,00402,54.0,80.73.2,EXCEED 72HRS-ST,68.0,34.109339,-118.316927,1
8,824,4359867912,2019-09-17,1028.0,NaN,NaN,CA,NaN,MERC,PA,SL,1222 POINT VIEW ST,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.053774,-118.369244,1
9,897,4359725355,2019-09-17,138.0,NaN,NaN,CA,NaN,JEEP,PA,GY,1000 39TH ST W,00500,55.0,80.56E4+,RED ZONE,93.0,34.015481,-118.291537,1


In [69]:
week_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week_names_dict = {index: name for index, name in enumerate(week_names)}

In [70]:
week_names_dict

{0: 'Monday',
 1: 'Tuesday',
 2: 'Wednesday',
 3: 'Thursday',
 4: 'Friday',
 5: 'Saturday',
 6: 'Sunday'}

In [71]:
df.loc[:,'Day of Week'] = df['Day of Week'].replace(week_names_dict)

In [72]:
df

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Day of Week
0,28,4361326712,2019-09-17,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.989412,-118.469024,Tuesday
1,112,4361286823,2019-09-17,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.962435,-118.290126,Tuesday
2,224,4361159515,2019-09-17,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.032120,-118.284957,Tuesday
3,473,4361237226,2019-09-17,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,33.994284,-118.474775,Tuesday
4,592,4360129196,2019-09-17,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.095217,-118.337472,Tuesday
5,618,4360059804,2019-09-17,1032.0,NaN,NaN,CA,NaN,CHEV,PA,BK,250 CORONADO ST S,00452,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.071393,-118.274392,Tuesday
6,679,4359986643,2019-09-17,1054.0,NaN,NaN,CA,NaN,HOND,PA,WT,15211 KITTRIDGE ST,350A,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.190290,-118.464223,Tuesday
7,694,4359985136,2019-09-17,1057.0,NaN,NaN,CA,NaN,TOYT,PA,BL,2131 CANYON DR,00402,54.0,80.73.2,EXCEED 72HRS-ST,68.0,34.109339,-118.316927,Tuesday
8,824,4359867912,2019-09-17,1028.0,NaN,NaN,CA,NaN,MERC,PA,SL,1222 POINT VIEW ST,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.053774,-118.369244,Tuesday
9,897,4359725355,2019-09-17,138.0,NaN,NaN,CA,NaN,JEEP,PA,GY,1000 39TH ST W,00500,55.0,80.56E4+,RED ZONE,93.0,34.015481,-118.291537,Tuesday


In [73]:
def nearby_coords(df, lat, lon, radius=0.0001, st_clean_only=True):
    try:
        if st_clean_only:
            return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude']
                      .between(lon-radius, lon+radius) & (df['Violation Description'] == 'NO PARK/STREET CLEAN')]
        else:
            return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude'].between(lon-radius, lon+radius)]
    except:
        return

In [74]:
def nearby_coords(df, lat, lon, radius=0.0001, st_clean_only=True):
    if st_clean_only:
        return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude']
                  .between(lon-radius, lon+radius) & (df['Violation Description'] == 'NO PARK/STREET CLEAN')]
    else:
        return df[df['Latitude'].between(lat-radius, lat+radius) & df['Longitude'].between(lon-radius, lon+radius)]


In [76]:
nearby_coords(df, 34.039664,-118.322450, 0.002)

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Day of Week
24231,2415443,4331631043,2018-05-03,1052.0,NaN,NaN,AZ,NaN,FORD,TK,WT,3615 21ST ST W,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.037979,-118.323801,Thursday
36198,3601480,4318561332,2017-10-19,832.0,NaN,NaN,NV,NaN,NISS,PA,GY,1823 4TH AVE,00147,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.040634,-118.321266,Thursday
36199,3601481,4318561321,2017-10-19,828.0,NaN,NaN,CA,NaN,NISS,PA,RD,1726 5TH AVE,00147,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.041558,-118.322458,Thursday
44906,4471143,4310082442,2017-05-31,1041.0,NaN,NaN,AL,NaN,NISS,PA,WT,2025 6TH AVENUE,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.038695,-118.323634,Wednesday
78314,7796016,4272088726,2015-12-02,1010.0,NaN,NaN,GA,NaN,CHEV,PA,GY,2000 4TH AVE,00185,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.038934,-118.321259,Wednesday
87553,8706345,4262600580,2015-07-02,1017.0,NaN,NaN,CA,NaN,CHEV,PA,RD,1925 5TH AVE,00171,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,34.039664,-118.322450,Thursday


In [77]:
nearby_coords(df, 33.989412,-118.469024, .01)[df['Location'].str.contains('WINDWARD') 
                        & (df['Day of Week'] =='Tuesday')][['Location', 'Issue time']].sort_values('Issue time')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Location,Issue time
58012,46 WINDWARD AVE,1106.0
48124,32 WINDWARD AVENUE,1107.0
60772,52 WINDWARD AVE,1108.0
8345,56 WINDWARD AVE,1109.0
75290,54 WINDWARD AVENUE,1110.0
7280,64 WINDWARD AVE,1111.0
59393,40 WINDWARD AVE,1114.0
78686,46 WINDWARD AVE E,1117.0
42351,66 WINDWARD AVE,1118.0
22779,36 WINDWARD AVE,1132.0


In [87]:
df_par=nearby_coords(df, 33.989412,-118.469024, .01)[df['Location'].str.contains('WINDWARD') 
                        & (df['Day of Week'] =='Tuesday')][['Location', 'Issue time','Latitude','Longitude']].sort_values('Issue time')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [88]:
df_par

,Location,Issue time,Latitude,Longitude
58012,46 WINDWARD AVE,1106.0,33.987283,-118.472913
48124,32 WINDWARD AVENUE,1107.0,33.987176,-118.473087
60772,52 WINDWARD AVE,1108.0,33.987328,-118.472838
8345,56 WINDWARD AVE,1109.0,33.987359,-118.472788
75290,54 WINDWARD AVENUE,1110.0,33.987344,-118.472813
7280,64 WINDWARD AVE,1111.0,33.987420,-118.472689
59393,40 WINDWARD AVE,1114.0,33.987237,-118.472987
78686,46 WINDWARD AVE E,1117.0,33.987283,-118.472913
42351,66 WINDWARD AVE,1118.0,33.987435,-118.472664
22779,36 WINDWARD AVE,1132.0,33.987207,-118.473037


In [94]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
mapbox_access_token = ''
import json
import geopandas 
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon

In [96]:
#"Latitude","Longitude"
geometry = [Point(xy) for xy in zip(df_par['Longitude'], df_par['Latitude'])]

# Coordinate reference system : WGS84
crs = {'epsg:4326'}

# Creating a Geographic data frame 
gdf = geopandas.GeoDataFrame(df_par, crs=crs, geometry=geometry)

In [97]:
gdf

,Location,Issue time,Latitude,Longitude,geometry
58012,46 WINDWARD AVE,1106.0,33.987283,-118.472913,POINT (-118.47291 33.98728)
48124,32 WINDWARD AVENUE,1107.0,33.987176,-118.473087,POINT (-118.47309 33.98718)
60772,52 WINDWARD AVE,1108.0,33.987328,-118.472838,POINT (-118.47284 33.98733)
8345,56 WINDWARD AVE,1109.0,33.987359,-118.472788,POINT (-118.47279 33.98736)
75290,54 WINDWARD AVENUE,1110.0,33.987344,-118.472813,POINT (-118.47281 33.98734)
7280,64 WINDWARD AVE,1111.0,33.987420,-118.472689,POINT (-118.47269 33.98742)
59393,40 WINDWARD AVE,1114.0,33.987237,-118.472987,POINT (-118.47299 33.98724)
78686,46 WINDWARD AVE E,1117.0,33.987283,-118.472913,POINT (-118.47291 33.98728)
42351,66 WINDWARD AVE,1118.0,33.987435,-118.472664,POINT (-118.47266 33.98743)
22779,36 WINDWARD AVE,1132.0,33.987207,-118.473037,POINT (-118.47304 33.98721)


In [98]:
from_json = gdf.to_json()
geoJSON = json.loads(from_json)

In [99]:
geoJSON

{'type': 'FeatureCollection',
 'features': [{'id': '58012',
   'type': 'Feature',
   'properties': {'Issue time': 1106.0,
    'Latitude': 33.98728266775373,
    'Location': '46 WINDWARD AVE',
    'Longitude': -118.4729126673601},
   'geometry': {'type': 'Point',
    'coordinates': [-118.4729126673601, 33.98728266775373]}},
  {'id': '48124',
   'type': 'Feature',
   'properties': {'Issue time': 1107.0,
    'Latitude': 33.98717623163849,
    'Location': '32 WINDWARD AVENUE',
    'Longitude': -118.47308676882587},
   'geometry': {'type': 'Point',
    'coordinates': [-118.47308676882587, 33.98717623163849]}},
  {'id': '60772',
   'type': 'Feature',
   'properties': {'Issue time': 1108.0,
    'Latitude': 33.987328338345336,
    'Location': '52 WINDWARD AVE',
    'Longitude': -118.47283792830754},
   'geometry': {'type': 'Point',
    'coordinates': [-118.47283792830754, 33.987328338345336]}},
  {'id': '8345',
   'type': 'Feature',
   'properties': {'Issue time': 1109.0,
    'Latitude': 33.98

In [100]:
pts=[]#list of points defining boundaries of polygons
for  feature in geoJSON['features']:
    if feature['geometry']['type']=='Point':
        pts.extend(feature['geometry']['coordinates'])    
        #pts.append([None, None])#mark the end of a polygon   

    elif feature['geometry']['type']=='MultiPolygon':
        for polyg in feature['geometry']['coordinates']:
            pts.extend(polyg[0])
            pts.append([None, None])#end of polygon
    else: raise ValueError("geometry type irrelevant for map") 

In [101]:
X = []
Y = []
for i in range(len(pts)):        
    if i % 2 == 0:
        X.append(pts[i])
    else:
        Y.append(pts[i])
        

In [103]:
X[:10]

[-118.4729126673601,
 -118.47308676882587,
 -118.47283792830754,
 -118.47278830890212,
 -118.47281327346654,
 -118.47268882197181,
 -118.47298728232916,
 -118.47291255251461,
 -118.47266395021502,
 -118.47303702559556]

In [105]:
data = [
go.Scattermapbox(
    lat=Y,
    lon=X,
    mode='markers',
    marker=dict(size= 3,color = 'red',opacity = .8))
    ]

layout = go.Layout(autosize=True,
                   mapbox = dict(center= dict(lat=33,     #Set center at Flataklocken
                                 lon=-118),        
                                 accesstoken=mapbox_access_token,
                                 zoom=4,
                                 style='light',
                               ),
                   
                    width=700,
                    height=600,
                    title = 'LA')


fig = go.Figure(data=data, layout=layout)
plot(fig, filename='LA Citation Map test')

C:\ProgramData\Anaconda3\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `LA Citation Map test` didn't end with .html. Adding .html to the end of your file.



'LA Citation Map test.html'